## Overview

It's time to revisit the way table coordinates work.  In particular, the new design should support the following:

* Columns and rows can be inserted into table coordinates.
    * Implies that embedded coordinate system ranges can be altered after construction.
* Table data is exportable.
    * We will store data as a heteregenous matrix of values.  The values in merged cells will be repeated.
* Better table coordinate cell access.
    * Allow regular data-table-like cell indexing.
        * This will make it easier to do alternating row colors, grouped styles, etc.
    * We might-or-might-not get rid of cell(), row(), and column() methods.
* Table-aware plot embedding.
    * It should be possible to embed a bar plot or a line plot simply by assigning a special "formatter" to a row / column.
    * The layout of the embedded plot would be affected by cell spacing, gaps, padding, etc.
    * The tricky part will be handling formatters that get overridden / are discontiguous.
* Efficiency.
    * Currently, we store a heavyweight `Cell` data structure for every cell in the table, with lots of redundant information.
        * This is inefficient for visualizations that don't need to control every little thing, like `matrix` vis.
        * Store cell properties using homogeneous matrix data structures.
        * We can store the row heights and column widths globally.
    * Merged cells are stored using extra `Cell` data structures that have pointers to their "parent" cells, but it's undefined what happens when merging merged cells.
        * Handle merging using "merge groups" instead.
        * Allow merged cells to be split apart by subsequent merges.

In [1]:
import numpy
import toyplot.data

In [2]:
data = toyplot.data.Table()
data["a"] = numpy.array(["Foo-BAZ-46", "Bar-BAZ-46", "Foo-BAZ-23", "Bar-BAZ-23", "Foo-BLAH", "Bar-BLAH"])
data["b"] = numpy.array([0.381, 0.142, 0.473, 0.234, 0.165, 0.186])

In [3]:
canvas, table = toyplot.table(data, width=600, height=250)

a b Foo-BAZ-46 0 . 381 Bar-BAZ-46 0 . 142 Foo-BAZ-23 0 . 473 Bar-BAZ-23 0 . 234 Foo-BLAH 0 . 165 Bar-BLAH 0 . 186

In [4]:
colormap = toyplot.color.CategoricalMap()

canvas = toyplot.Canvas(width=600, height=250)
table = canvas.table(rows=6, columns=4, label="Quarterly Report")
table.grid.hlines[...] = "single"
table.grid.vlines[...] = "single"
table.cells.column(0).width = 50
table.cells.column(1).data = data["a"]
table.cells.column(1).width = 90
table.cells.column(3).data = data["b"]
table.cells.column(3).width = 55
table.cells.column(3).format = toyplot.format.FloatFormatter(format="{:.2f}s")
table.cells.cell(0,0,rowspan=2).merge().data = "Red"
table.cells.cell(2,0,rowspan=2).merge().data = "Green"
table.cells.cell(4,0,rowspan=2).merge().data = "Blue"
#axes = table.cell(0, 2, rowspan=6).axes(show=True, yshow=False, padding=0, xlabel="Seconds")
#axes.x.spine.show = False
#axes.bars(data["b"][::-1], along="y", style={"stroke-width":8}, color=(numpy.array([0,1,0,1,0,1])[::-1], colormap));

AttributeError: 'CellReference' object has no attribute 'merge'

Foo-BAZ-46 0.381 Bar-BAZ-46 0.142 Foo-BAZ-23 0.473 Bar-BAZ-23 0.234 Foo-BLAH 0.165 Bar-BLAH 0.186 Quarterly Report